# Opentrons Notebook

This notebook is for learning how to use the opentrons python API to create protocols for automation.

## Imports
Install the opentrons api and import.

In [1]:
from opentrons import protocol_api

## Metadata
Fill in your personal deails and method description.

In [2]:
# metadata
metadata = {
    'protocolName': 'Heat Shock',
    'author': 'Thom Booth <thoboo@biosustain.dtu.dk>',
    'description': 'Protocol for the transformation of chemical competent E.coli.',
    'apiLevel': '2.12'
}

## Protocol
This function will run the protocol.

### Setting up labware, modules and instruments

Specify the labware, pippettes and finally the protocol itself. 

For each piece of labware, use: `protocol.load_labware('name_of_labware', int(slot_number))`. 
For a list of default labware, see: https://labware.opentrons.com/. Custom labware objects can also be generated if needed.

For heatshock, we also need to add our heat block. This is a special module so must be added seperately from other labware. Use: use: `protocol.load_module()`. More info on modules can be found here: https://docs.opentrons.com/v2/new_modules.html. You also need to load labware to your module! See: `module.load_labware()`.

For the left and right pippette, use: `protocol.load_instrument('name_of_instrument', 'left_or_right', tip_racks=[name_of_tiprack])`.

### Creating the Commands
Now all we need to do is specify commands in the correct order.

In this example, we will:
1. Set the temperature and wait for the temperature to reach 42 degrees;
2. Transfer 2 ul of DNA from the DNA plate to the E.coli plates and wait for 10 mins;
3. Transfer 22 ul of E.coli + DNA to the heatblock and wait for 45 seconds;
4. Transfer 20 ul of E.coli + DNA to the E.coli plate and wait for 1 min;
5. Transfer 150 ul LB from media tube to each well of the E.coli plate.

In [3]:
def run(protocol: protocol_api.ProtocolContext):
    # labware
    dna_plate = protocol.load_labware('biorad_96_wellplate_200ul_pcr', '1')
    ecoli_plate = protocol.load_labware('biorad_96_wellplate_200ul_pcr', '2')
    media_rack = protocol.load_labware('!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!', '4')
    tiprack_20 = protocol.load_labware('opentrons_96_tiprack_20ul', '10')
    tiprack_1000 = protocol.load_labware('opentrons_96_tiprack_1000ul', '9')
    # modules
    heat_block = protocol.load_module('temperature module', 3)
    heat_plate = heat_block.load_labware('opentrons_96_aluminumblock_generic_pcr_strip_200ul')
    # pipettes
    left_pipette = protocol.load_instrument('p20_multi_gen2', 'left', tip_racks=[tiprack_20])
    right_pipette = protocol.load_instrument('p1000_multi_gen2', 'right', tip_racks=[tiprack_1000])
    # commands
    #Step 1 - Get the heatblock to temp and wait for user
    heat_block.set_temperature(42) #Note new version waits automatically.
    
    #Step 2 - Add DNA to E.coli and wait 10 mins
    left.pick_up_tip()
    left.aspirate(2, dna_plate['A1'])
    left.dispense(2, ecoli_plate['A1'].bottom())
    pipette.mix(3, 10)
    left.drop_tip()
    protocol.delay(minutes=10)
    
    #Step 3 - Transfer to headblock and incubate for 45s
    left.pick_up_tip()
    left.aspirate(21, ecoli_plate['A1'])
    left.dispense(21, heat_plate['A1'])
    left.drop_tip()
    protocol.delay(seconds=45)
    
    #Step 4 - Transfer back to block
    left.pick_up_tip()
    left.aspirate(20, heat_plate['A1'])
    left.dispense(20, ecoli_plate['A1'])
    left.drop_tip()
    
    #Step 5 - Add LB
    row = ['A1','A2','A3','A4','A5','A6','A7','A8']
    for well in row:
        right.pick_up_tip()
        left.aspirate(1000, media_plate['A1'])
        left.dispense(150, ecoli_plate[well])
        left.drop_tip()